# Machine learning homework 6

## 0. Preparation

### 0.1 Import required librarys

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### 0.2 Read data from files

In [ ]:
circle = np.loadtxt('data/circle.txt', delimiter=',')
moon = np.loadtxt('data/moon.txt', delimiter=',')

### 0.3 Take a look at the data

In [ ]:
fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(121)
ax.set_title('circle')
ax.scatter(circle[:, 0], circle[:, 1])
ax = fig.add_subplot(122)
ax.set_title('moon')
ax.scatter(moon[:, 0], moon[:, 1])
plt.show()